In [23]:
import os
import cv2
import numpy as np
import math  

In [24]:
# 1. 1. 이미지 전처리, 노이즈 제거 (포함 시 추가 점수)

def gaussian_kernel(size, sigma):
    """2D Gaussian Kernel 생성"""
    kernel = np.zeros((size, size), dtype=np.float32)
    center = size // 2
    for i in range(size):
        for j in range(size):
            diff = (i - center) ** 2 + (j - center) ** 2
            kernel[i, j] = np.exp(-diff / (2 * sigma ** 2))
    kernel /= np.sum(kernel)  # 정규화
    print(f"Gaussian Kernel 생성 완료 (크기: {size}, sigma: {sigma})")
    return kernel

def apply_gaussian_filter(image, kernel):
    """이미지에 Gaussian Filter 적용"""
    height, width, channels = image.shape
    k_size = kernel.shape[0]
    pad = k_size // 2
    padded_image = np.pad(image, ((pad, pad), (pad, pad), (0, 0)), mode='constant', constant_values=0)
    filtered_image = np.zeros_like(image)

    print(f"이미지 크기: {image.shape}, 패딩 추가된 크기: {padded_image.shape}")

    # Convolution 수행
    for y in range(height):
        if y % 100 == 0:  # 100줄마다 진행 상황 표시
            print(f"처리 중... {y}/{height} 줄 완료")
        for x in range(width):
            for c in range(channels):  # 채널별 처리
                region = padded_image[y:y + k_size, x:x + k_size, c]
                filtered_image[y, x, c] = np.sum(region * kernel)
    print(f"Gaussian Filter 적용 완료")
    return filtered_image

# 이미지가 저장된 폴더 경로를 지정하세요
folder_path = 'C:/Users/Jaehyun Byun/PycharmProjects/Panoramic/PanormicStitching/0. Image Input'

# 폴더 내의 모든 파일 목록 가져오기
file_names = os.listdir(folder_path)
print(f"폴더에서 {len(file_names)}개의 파일을 찾았습니다.")

# 이미지 파일만 필터링하기 (확장자별)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
image_files = [file for file in file_names if os.path.splitext(file)[1].lower() in image_extensions]
print(f"이미지 파일로 필터링된 파일 수: {len(image_files)}")

# Gaussian 커널 생성
kernel_size = 3  # 커널 크기 (홀수로 설정)
sigma = 1  # 표준 편차
gaussian_k = gaussian_kernel(kernel_size, sigma)

# 이미지 불러오기 및 Gaussian Smoothing Filter 적용
filtered_images = []
for idx, file_name in enumerate(image_files):
    print(f"{idx+1}/{len(image_files)}: {file_name} 처리 중...")
    img_path = os.path.join(folder_path, file_name)
    img = cv2.imread(img_path)
    if img is not None:
        print(f"이미지 로드 성공: {file_name}, 크기: {img.shape}")
        filtered_img = apply_gaussian_filter(img, gaussian_k)
        filtered_images.append((file_name, filtered_img))
    else:
        print(f"이미지를 불러올 수 없습니다: {img_path}")

# 필터링된 이미지를 저장하거나 활용하기
output_folder = '1. Image Filtering'  # 결과 이미지를 저장할 폴더 이름
os.makedirs(output_folder, exist_ok=True)
print(f"출력 폴더 생성 완료: {output_folder}")

for idx, (file_name, filtered_img) in enumerate(filtered_images):
    output_path = os.path.join(output_folder, file_name)
    cv2.imwrite(output_path, filtered_img)
    print(f"{idx+1}/{len(filtered_images)}: 필터링된 이미지를 저장했습니다: {output_path}")

print("1. Image Filtering 단계가 완료되었습니다.")

폴더에서 11개의 파일을 찾았습니다.
이미지 파일로 필터링된 파일 수: 10
Gaussian Kernel 생성 완료 (크기: 3, sigma: 1)
1/10: testimg01.png 처리 중...
이미지 로드 성공: testimg01.png, 크기: (640, 480, 3)
이미지 크기: (640, 480, 3), 패딩 추가된 크기: (642, 482, 3)
처리 중... 0/640 줄 완료
처리 중... 100/640 줄 완료
처리 중... 200/640 줄 완료
처리 중... 300/640 줄 완료
처리 중... 400/640 줄 완료
처리 중... 500/640 줄 완료
처리 중... 600/640 줄 완료
Gaussian Filter 적용 완료
2/10: testimg02.png 처리 중...
이미지 로드 성공: testimg02.png, 크기: (640, 480, 3)
이미지 크기: (640, 480, 3), 패딩 추가된 크기: (642, 482, 3)
처리 중... 0/640 줄 완료
처리 중... 100/640 줄 완료
처리 중... 200/640 줄 완료
처리 중... 300/640 줄 완료
처리 중... 400/640 줄 완료
처리 중... 500/640 줄 완료
처리 중... 600/640 줄 완료
Gaussian Filter 적용 완료
3/10: testimg03.png 처리 중...
이미지 로드 성공: testimg03.png, 크기: (640, 480, 3)
이미지 크기: (640, 480, 3), 패딩 추가된 크기: (642, 482, 3)
처리 중... 0/640 줄 완료
처리 중... 100/640 줄 완료
처리 중... 200/640 줄 완료
처리 중... 300/640 줄 완료
처리 중... 400/640 줄 완료
처리 중... 500/640 줄 완료
처리 중... 600/640 줄 완료
Gaussian Filter 적용 완료
4/10: testimg04.png 처리 중...
이미지 로드 성공: testimg04.png, 크기

In [25]:
# 2. 코너 포인트 찾기 (필수 구현)

def to_grayscale(image):
    """이미지를 그레이스케일로 변환"""
    print(f"[DEBUG] Converting image to grayscale. Image shape: {image.shape}")
    if len(image.shape) == 3 and image.shape[2] == 3:
        # RGB 가중치를 사용한 그레이스케일 변환
        grayscale = 0.299 * image[:, :, 2] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 0]
        grayscale = grayscale.astype(np.float32)
        print(f"[DEBUG] Grayscale conversion using RGB weights completed.")
    else:
        # 이미 그레이스케일이면 그대로 float32로 변환
        grayscale = image.astype(np.float32)
        print(f"[DEBUG] Image is already grayscale.")
    print("그레이스케일 변환 완료")
    return grayscale

def compute_image_gradients(image):
    """Sobel 필터를 사용하여 이미지 그라디언트 계산"""
    print(f"[DEBUG] Computing image gradients. Image shape: {image.shape}")
    # Sobel 커널 정의
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], dtype=np.float32)
    sobel_y = np.array([[-1, -2, -1],
                        [ 0,  0,  0],
                        [ 1,  2,  1]], dtype=np.float32)
    
    k_size = sobel_x.shape[0]
    pad = k_size // 2
    padded_image = np.pad(image, ((pad, pad), (pad, pad)), mode='constant', constant_values=0)
    print(f"[DEBUG] Padded image for gradients: {padded_image.shape}")
    
    I_x = np.zeros_like(image, dtype=np.float32)
    I_y = np.zeros_like(image, dtype=np.float32)
    
    height, width = image.shape
    print("그라디언트 계산 시작")
    for y in range(height):
        if y % max(1, height // 10) == 0:  # 10% 진행 시마다 표시
            print(f"[DEBUG] Gradient processing row {y+1}/{height} ({(y+1)/height*100:.1f}%)")
        for x in range(width):
            region = padded_image[y:y + k_size, x:x + k_size]
            I_x[y, x] = np.sum(region * sobel_x)
            I_y[y, x] = np.sum(region * sobel_y)
            # 디버그 용으로 첫 픽셀의 결과값 확인
            if y == 0 and x == 0:
                print(f"[DEBUG] First gradient values: I_x={I_x[y, x]}, I_y={I_y[y, x]}")
    print("그라디언트 계산 완료")
    return I_x, I_y

def compute_harris_response(I_x, I_y, kernel, k=0.04):
    """Harris 응답 계산"""
    print("[DEBUG] Computing Harris response.")
    # Ixx, Iyy, Ixy 계산
    Ixx = I_x ** 2
    Iyy = I_y ** 2
    Ixy = I_x * I_y
    print(f"[DEBUG] Computed Ixx, Iyy, Ixy. Sample values: Ixx[0,0]={Ixx[0,0]}, Iyy[0,0]={Iyy[0,0]}, Ixy[0,0]={Ixy[0,0]}")
    
    # 가우시안 필터가 이미 있으므로, 필요하다면 그대로 적용
    print("Harris 응답을 위한 Ixx, Iyy, Ixy에 Gaussian 필터 적용")
    Ixx = apply_gaussian_filter(Ixx[..., np.newaxis], kernel)[..., 0]
    Iyy = apply_gaussian_filter(Iyy[..., np.newaxis], kernel)[..., 0]
    Ixy = apply_gaussian_filter(Ixy[..., np.newaxis], kernel)[..., 0]
    print(f"[DEBUG] Applied Gaussian filter to Ixx, Iyy, Ixy. Sample values: Ixx[0,0]={Ixx[0,0]}, Iyy[0,0]={Iyy[0,0]}, Ixy[0,0]={Ixy[0,0]}")
    
    # Harris 응답 계산
    print("Harris 응답 계산 중")
    det_M = Ixx * Iyy - Ixy ** 2
    trace_M = Ixx + Iyy
    R = det_M - k * (trace_M ** 2)
    print(f"[DEBUG] Computed Harris response R. Sample R[0,0]={R[0,0]}")
    print("Harris 응답 계산 완료")
    return R

def find_corners(R, threshold_ratio=0.01, window_size=3):
    """Harris 응답에서 코너 포인트 찾기"""
    print("[DEBUG] Finding corners in Harris response.")
    threshold = threshold_ratio * np.max(R)
    print(f"코너 임계값 설정: {threshold}")
    corners = []
    height, width = R.shape
    offset = window_size // 2
    
    for y in range(offset, height - offset):
        if y % max(1, height // 10) == 0:  # 10% 진행 시마다 표시
            print(f"[DEBUG] Corner detection processing row {y+1}/{height} ({(y+1)/height*100:.1f}%)")
        for x in range(offset, width - offset):
            window = R[y - offset:y + offset + 1, x - offset:x + offset + 1]
            # 윈도우 내 최대값이고 threshold 초과인 경우
            if R[y, x] == np.max(window) and R[y, x] > threshold:
                corners.append((x, y))
                # 디버그: 앞 부분만 코너 위치 출력
                if len(corners) <= 5:
                    print(f"[DEBUG] Corner found at (x={x}, y={y}) with R={R[y, x]}")
    print(f"총 {len(corners)}개의 코너 포인트 발견")
    return corners

def mark_corners(image, corners, color=(0, 0, 255)):
    """코너 포인트를 이미지에 표시"""
    print("[DEBUG] Marking corners on image.")
    marked_image = image.copy()
    for idx, (x, y) in enumerate(corners):
        cv2.circle(marked_image, (x, y), radius=3, color=color, thickness=1)
        if idx < 5:  # 처음 5개 코너 포인트만 출력
            print(f"[DEBUG] Drawing circle at (x={x}, y={y})")
    return marked_image

# Corner Detection 단계
corner_output_folder = '2. Corner Detection'
os.makedirs(corner_output_folder, exist_ok=True)
print(f"출력 폴더 생성 완료: {corner_output_folder}")

for idx, (file_name, filtered_img) in enumerate(filtered_images):
    print(f"{idx+1}/{len(filtered_images)}: {file_name}에서 코너 포인트 검출 중...")
    grayscale = to_grayscale(filtered_img)
    I_x, I_y = compute_image_gradients(grayscale)
    R = compute_harris_response(I_x, I_y, gaussian_k, k=0.06)
    corners = find_corners(R, threshold_ratio=0.005, window_size=5)
    marked_img = mark_corners(filtered_img, corners, color=(0, 0, 255))
    output_path = os.path.join(corner_output_folder, f"corners_{file_name}")
    cv2.imwrite(output_path, marked_img)
    corners_dict = {} if idx == 0 else corners_dict
    corners_dict[file_name] = corners
    print(f"{idx+1}/{len(filtered_images)}: 코너가 표시된 이미지를 저장했습니다: {output_path}")

print("2. Corner Detection 단계가 완료되었습니다.")

출력 폴더 생성 완료: 2. Corner Detection
1/10: testimg01.png에서 코너 포인트 검출 중...
[DEBUG] Converting image to grayscale. Image shape: (640, 480, 3)
[DEBUG] Grayscale conversion using RGB weights completed.
그레이스케일 변환 완료
[DEBUG] Computing image gradients. Image shape: (640, 480)
[DEBUG] Padded image for gradients: (642, 482)
그라디언트 계산 시작
[DEBUG] Gradient processing row 1/640 (0.2%)
[DEBUG] First gradient values: I_x=570.989013671875, I_y=570.989013671875
[DEBUG] Gradient processing row 65/640 (10.2%)
[DEBUG] Gradient processing row 129/640 (20.2%)
[DEBUG] Gradient processing row 193/640 (30.2%)
[DEBUG] Gradient processing row 257/640 (40.2%)
[DEBUG] Gradient processing row 321/640 (50.2%)
[DEBUG] Gradient processing row 385/640 (60.2%)
[DEBUG] Gradient processing row 449/640 (70.2%)
[DEBUG] Gradient processing row 513/640 (80.2%)
[DEBUG] Gradient processing row 577/640 (90.2%)
그라디언트 계산 완료
[DEBUG] Computing Harris response.
[DEBUG] Computed Ixx, Iyy, Ixy. Sample values: Ixx[0,0]=326028.46875, Iyy[0,0]

In [26]:
import random
from tqdm import tqdm  # 진행 상황 표시를 위한 라이브러리

# 3. Point Matching, 4. RANSAC, 5. Homography 추가
def extract_descriptors(image, corners, patch_size=11):
    """코너 포인트 주변의 패치를 디스크립터로 추출"""
    print("[DEBUG] Extracting descriptors from corners.")
    descriptors = {}
    offset = patch_size // 2
    padded_image = np.pad(image, ((offset, offset), (offset, offset), (0,0)), mode='constant', constant_values=0)
    
    for (x, y) in corners:
        if y < offset or y >= image.shape[0] - offset or x < offset or x >= image.shape[1] - offset:
            continue  # 패치가 이미지 밖으로 벗어나지 않도록
        patch = padded_image[y:y + patch_size, x:x + patch_size]
        patch_gray = to_grayscale(patch)
        descriptor = patch_gray.flatten()
        norm = np.linalg.norm(descriptor)
        if norm == 0:
            continue  # 제로 벡터는 제외
        descriptor = descriptor / norm  # 정규화
        descriptors[(x, y)] = descriptor
    print(f"[DEBUG] Extracted {len(descriptors)} descriptors.")
    return descriptors

def match_descriptors(desc1, desc2, ratio_threshold=0.75):
    """두 이미지의 디스크립터 매칭 (최소 거리 기준, Lowe의 비율 테스트 적용)"""
    print("[DEBUG] Matching descriptors between two images.")
    matches = []
    desc2_keys = list(desc2.keys())
    desc2_values = np.array(list(desc2.values()))
    
    for key1, descriptor1 in tqdm(desc1.items(), desc="Matching Descriptors"):
        distances = np.linalg.norm(desc2_values - descriptor1, axis=1)
        if len(distances) < 2:
            continue
        sorted_idx = np.argsort(distances)
        best_idx = sorted_idx[0]
        second_best_idx = sorted_idx[1]
        if distances[best_idx] < ratio_threshold * distances[second_best_idx]:
            key2 = desc2_keys[best_idx]
            matches.append((key1, key2))
    print(f"[DEBUG] Found {len(matches)} good matches.")
    return matches

def visualize_matches(img1, img2, matches, output_path, max_matches=50):
    """매칭된 포인트를 시각화하여 이미지로 저장"""
    print("[DEBUG] Visualizing matches.")
    # 두 이미지를 좌우로 합침
    height1, width1, _ = img1.shape
    height2, width2, _ = img2.shape
    combined_height = max(height1, height2)
    combined_width = width1 + width2
    combined_image = np.zeros((combined_height, combined_width, 3), dtype=np.uint8)
    combined_image[:height1, :width1] = img1
    combined_image[:height2, width1:width1 + width2] = img2
    
    # 랜덤하게 매칭을 시각화
    for idx, ((x1, y1), (x2, y2)) in enumerate(matches[:max_matches]):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        cv2.circle(combined_image, (x1, y1), 3, color, -1)
        cv2.circle(combined_image, (x2 + width1, y2), 3, color, -1)
        cv2.line(combined_image, (x1, y1), (x2 + width1, y2), color, 1)
    
    cv2.imwrite(output_path, combined_image)
    print(f"[DEBUG] Match visualization saved to {output_path}")

def compute_homography(p1, p2):
    """호모그래피 행렬을 계산 (Direct Linear Transformation)"""
    A = []
    for (x1, y1), (x2, y2) in zip(p1, p2):
        A.append([-x1, -y1, -1, 0, 0, 0, x1 * x2, y1 * x2, x2])
        A.append([0, 0, 0, -x1, -y1, -1, x1 * y2, y1 * y2, y2])
    A = np.array(A)
    print("[DEBUG] Computing SVD for homography.")
    U, S, Vt = np.linalg.svd(A)
    H = Vt[-1].reshape(3, 3)
    H = H / H[2, 2]
    return H

def ransac_homography(matches, num_iterations=1000, threshold=5.0):
    """RANSAC 알고리즘을 사용하여 호모그래피 추정"""
    print("[DEBUG] Starting RANSAC for homography estimation.")
    max_inliers = []
    best_H = None
    for i in tqdm(range(num_iterations), desc="RANSAC Iterations"):
        # 랜덤하게 4개의 매칭 선택
        try:
            sampled_matches = random.sample(matches, 4)
        except ValueError as e:
            print(f"[ERROR] RANSAC: {e}")
            break
        p1 = [m[0] for m in sampled_matches]
        p2 = [m[1] for m in sampled_matches]
        H = compute_homography(p1, p2)
        
        # 모든 매칭에 대해 호모그래피 적용
        inliers = []
        for (pt1, pt2) in matches:
            x, y = pt1
            vec = np.array([x, y, 1])
            projected = H @ vec
            if projected[2] == 0:
                continue
            projected /= projected[2]
            x_proj, y_proj = projected[0], projected[1]
            distance = math.sqrt((x_proj - pt2[0]) ** 2 + (y_proj - pt2[1]) ** 2)
            if distance < threshold:
                inliers.append((pt1, pt2))
        
        if len(inliers) > len(max_inliers):
            max_inliers = inliers
            best_H = H
            print(f"[DEBUG] Iteration {i+1}: Found {len(inliers)} inliers.")
        
        # 조기 종료 조건 (충분한 인라이어를 찾은 경우)
        if len(max_inliers) > 0.8 * len(matches):
            print(f"[DEBUG] Early stopping at iteration {i+1} with {len(max_inliers)} inliers.")
            break
    
    print(f"[DEBUG] RANSAC completed with {len(max_inliers)} inliers out of {len(matches)} matches.")
    return best_H, max_inliers

def visualize_ransac_matches(img1, img2, inliers, output_path, max_inliers=50):
    """RANSAC 인라이어 매칭을 시각화하여 이미지로 저장"""
    print("[DEBUG] Visualizing RANSAC inlier matches.")
    height1, width1, _ = img1.shape
    height2, width2, _ = img2.shape
    combined_height = max(height1, height2)
    combined_width = width1 + width2
    combined_image = np.zeros((combined_height, combined_width, 3), dtype=np.uint8)
    combined_image[:height1, :width1] = img1
    combined_image[:height2, width1:width1 + width2] = img2
    
    for idx, ((x1, y1), (x2, y2)) in enumerate(inliers[:max_inliers]):
        color = (0, 255, 0)  # 인라이어는 녹색
        cv2.circle(combined_image, (x1, y1), 3, color, -1)
        cv2.circle(combined_image, (x2 + width1, y2), 3, color, -1)
        cv2.line(combined_image, (x1, y1), (x2 + width1, y2), color, 1)
    
    cv2.imwrite(output_path, combined_image)
    print(f"[DEBUG] RANSAC inlier match visualization saved to {output_path}")

def apply_homography(img, H, output_shape):
    """호모그래피를 적용하여 이미지를 변환"""
    print("[DEBUG] Applying homography to image.")
    height, width = output_shape
    transformed_img = np.zeros((height, width, 3), dtype=np.uint8)
    for y in tqdm(range(height), desc="Applying Homography"):
        for x in range(width):
            vec = np.array([x, y, 1])
            projected = H @ vec
            if projected[2] == 0:
                continue
            projected /= projected[2]
            src_x, src_y = projected[0], projected[1]
            src_x = int(round(src_x))
            src_y = int(round(src_y))
            if 0 <= src_x < img.shape[1] and 0 <= src_y < img.shape[0]:
                transformed_img[y, x] = img[src_y, src_x]
    return transformed_img

# 추가적인 저장을 위한 폴더 생성
point_matching_folder = '3. Point Matching'
ransac_folder = '4. RANSAC'
homography_folder = '5. Homography'
os.makedirs(point_matching_folder, exist_ok=True)
os.makedirs(ransac_folder, exist_ok=True)
os.makedirs(homography_folder, exist_ok=True)
print("[DEBUG] Created folders for Point Matching, RANSAC, and Homography.")

# Load filtered and corner-detected images
corner_folder = '2. Corner Detection'
file_names = os.listdir(corner_folder)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
corner_images = []

for file_name in file_names:
    if file_name.startswith("corners_") and os.path.splitext(file_name)[1].lower() in image_extensions:
        img_path = os.path.join(corner_folder, file_name)
        img = cv2.imread(img_path)
        if img is not None:
            original_name = file_name.replace("corners_", "")
            corner_images.append((original_name, img))

print(f"[DEBUG] Loaded {len(corner_images)} corner-detected images from '{corner_folder}'.")

# 이미지 쌍을 1-2, 2-3, ..., 9-10 형태로 처리
total_pairs = len(corner_images) - 1
print(f"[DEBUG] 총 {total_pairs}개의 이미지 쌍을 처리할 예정입니다.")

for i in tqdm(range(total_pairs), desc="Processing Image Pairs"):
    img1_name, img1 = corner_images[i]
    img2_name, img2 = corner_images[i + 1]
    print(f"\n[INFO] Processing image pair: {img1_name} & {img2_name} ({i+1}/{total_pairs})")
    
    # Extract corners from images
    filtered_img1_path = os.path.join('1. Image Filtering', img1_name)
    filtered_img2_path = os.path.join('1. Image Filtering', img2_name)
    filtered_img1 = cv2.imread(filtered_img1_path)
    filtered_img2 = cv2.imread(filtered_img2_path)
    
    corners1 = corners_dict.get(img1_name, [])
    corners2 = corners_dict.get(img2_name, [])
    
    print(f"[DEBUG] Number of corners in {img1_name}: {len(corners1)}")
    print(f"[DEBUG] Number of corners in {img2_name}: {len(corners2)}")
    
    # 디스크립터 추출
    desc1 = extract_descriptors(filtered_img1, corners1)
    desc2 = extract_descriptors(filtered_img2, corners2)
    
    print(f"[DEBUG] Number of descriptors in {img1_name}: {len(desc1)}")
    print(f"[DEBUG] Number of descriptors in {img2_name}: {len(desc2)}")
    
    # 매칭 수행
    matches = match_descriptors(desc1, desc2, ratio_threshold=0.75)
    print(f"[DEBUG] Number of matches between {img1_name} & {img2_name}: {len(matches)}")
    
    # 매칭 포인트가 충분한지 확인
    if len(matches) < 4:
        print(f"[WARNING] Not enough matches ({len(matches)}) between {img1_name} and {img2_name}. Skipping RANSAC and Homography.")
        continue  # 다음 이미지 쌍으로 넘어감
    
    # 매칭 시각화 저장
    match_image_path = os.path.join(point_matching_folder, f"matches_{img1_name}_vs_{img2_name}.jpg")
    visualize_matches(filtered_img1, filtered_img2, matches, match_image_path)
    
    # 매칭된 포인트를 텍스트 파일로 저장
    matches_txt_path = os.path.join(point_matching_folder, f"matches_{img1_name}_vs_{img2_name}.txt")
    with open(matches_txt_path, 'w') as f:
        for (pt1, pt2) in matches:
            f.write(f"{pt1[0]},{pt1[1]},{pt2[0]},{pt2[1]}\n")
    print(f"[DEBUG] Saved matches to {matches_txt_path}")
    
    # RANSAC을 사용한 호모그래피 추정
    H, inliers = ransac_homography(matches, num_iterations=1000, threshold=5.0)
    
    # 인라이어가 충분한지 확인
    if len(inliers) < 4:
        print(f"[WARNING] Not enough inliers ({len(inliers)}) after RANSAC for {img1_name} and {img2_name}. Skipping Homography.")
        continue  # 다음 이미지 쌍으로 넘어감
    
    # RANSAC 인라이어 시각화 저장
    ransac_image_path = os.path.join(ransac_folder, f"ransac_{img1_name}_vs_{img2_name}.jpg")
    visualize_ransac_matches(filtered_img1, filtered_img2, inliers, ransac_image_path)
    
    # 인라이어 포인트를 텍스트 파일로 저장
    inliers_txt_path = os.path.join(ransac_folder, f"inliers_{img1_name}_vs_{img2_name}.txt")
    with open(inliers_txt_path, 'w') as f:
        for (pt1, pt2) in inliers:
            f.write(f"{pt1[0]},{pt1[1]},{pt2[0]},{pt2[1]}\n")
    print(f"[DEBUG] Saved inliers to {inliers_txt_path}")
    
    # 호모그래피 행렬 저장
    homography_txt_path = os.path.join(homography_folder, f"homography_{img1_name}_to_{img2_name}.txt")
    with open(homography_txt_path, 'w') as f:
        for row in H:
            f.write(' '.join(map(str, row)) + '\n')
    print(f"[DEBUG] Saved homography matrix to {homography_txt_path}")
    
    # 호모그래피를 적용하여 이미지 정합 (예시로 img2를 img1에 맞춤)
    # 출력 이미지의 크기는 img1과 img2를 합친 크기로 설정
    output_width = img1.shape[1] + img2.shape[1]
    output_height = max(img1.shape[0], img2.shape[0])
    aligned_img2 = apply_homography(img2, H, (output_height, output_width))
    
    # 정합된 이미지 시각화 저장
    aligned_image_path = os.path.join(homography_folder, f"aligned_{img1_name}_to_{img2_name}.jpg")
    cv2.imwrite(aligned_image_path, aligned_img2)
    print(f"[DEBUG] Saved aligned image to {aligned_image_path}")
    
    # 파노라마 이미지 생성 (간단한 합성)
    print("[DEBUG] Creating panorama image.")
    panorama = img1.copy()
    for y in range(aligned_img2.shape[0]):
        for x in range(aligned_img2.shape[1]):
            if not np.array_equal(aligned_img2[y, x], [0, 0, 0]):
                if y < panorama.shape[0] and x < panorama.shape[1]:
                    panorama[y, x] = aligned_img2[y, x]
                else:
                    # 이미지 크기 확장
                    if y >= panorama.shape[0]:
                        panorama = np.pad(panorama, ((0, y - panorama.shape[0] +1), (0, 0), (0,0)), mode='constant', constant_values=0)
                    if x >= panorama.shape[1]:
                        panorama = np.pad(panorama, ((0, 0), (0, x - panorama.shape[1] +1), (0,0)), mode='constant', constant_values=0)
                    panorama[y, x] = aligned_img2[y, x]
    
    # 파노라마 이미지 저장
    panorama_path = os.path.join(homography_folder, f"panorama_{img1_name}_and_{img2_name}.jpg")
    cv2.imwrite(panorama_path, panorama)
    print(f"[DEBUG] Saved panorama image to {panorama_path}")

print("모든 추가 작업이 완료되었습니다.")

[DEBUG] Created folders for Point Matching, RANSAC, and Homography.
[DEBUG] Loaded 10 corner-detected images from '2. Corner Detection'.
[DEBUG] 총 9개의 이미지 쌍을 처리할 예정입니다.


Processing Image Pairs:   0%|                                                                                                | 0/9 [00:00<?, ?it/s]


[INFO] Processing image pair: testimg01.png & testimg02.png (1/9)
[DEBUG] Number of corners in testimg01.png: 286
[DEBUG] Number of corners in testimg02.png: 109
[DEBUG] Extracting descriptors from corners.
[DEBUG] Converting image to grayscale. Image shape: (11, 11, 3)
[DEBUG] Grayscale conversion using RGB weights completed.
그레이스케일 변환 완료
[DEBUG] Converting image to grayscale. Image shape: (11, 11, 3)
[DEBUG] Grayscale conversion using RGB weights completed.
그레이스케일 변환 완료
[DEBUG] Converting image to grayscale. Image shape: (11, 11, 3)
[DEBUG] Grayscale conversion using RGB weights completed.
그레이스케일 변환 완료
[DEBUG] Converting image to grayscale. Image shape: (11, 11, 3)
[DEBUG] Grayscale conversion using RGB weights completed.
그레이스케일 변환 완료
[DEBUG] Converting image to grayscale. Image shape: (11, 11, 3)
[DEBUG] Grayscale conversion using RGB weights completed.
그레이스케일 변환 완료
[DEBUG] Converting image to grayscale. Image shape: (11, 11, 3)
[DEBUG] Grayscale conversion using RGB weights comple



atching Descriptors: 100%|███████████████████████████████████████████████████████████████████████████████████| 282/282 [00:00<00:00, 52751.48it/s]

[DEBUG] Found 65 good matches.
[DEBUG] Number of matches between testimg01.png & testimg02.png: 65
[DEBUG] Visualizing matches.
[DEBUG] Match visualization saved to 3. Point Matching\matches_testimg01.png_vs_testimg02.png.jpg
[DEBUG] Saved matches to 3. Point Matching\matches_testimg01.png_vs_testimg02.png.txt
[DEBUG] Starting RANSAC for homography estimation.


...

[DEBUG] Computing SVD for homography.
[DEBUG] Iteration 1: Found 4 inliers.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Iteration 22: Found 5 inliers.
[DEBUG] Computing SVD for homography.
[DEBUG] Computing SVD for homography.
[DEBUG] Com








































pplying Homography: 100%|██████████████████████████████████████████████████████████████████████████████████████| 640/640 [00:03<00:00, 165.56it/s]

[DEBUG] Saved aligned image to 5. Homography\aligned_testimg09.png_to_testimg10.png.jpg
[DEBUG] Creating panorama image.


Processing Image Pairs: 100%|████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:00<00:00,  6.67s/it]

[DEBUG] Saved panorama image to 5. Homography\panorama_testimg09.png_and_testimg10.png.jpg
모든 추가 작업이 완료되었습니다.


In [38]:
import re

def load_homography(output_folder, pair_name):
    """호모그래피 행렬을 .txt 파일에서 로드합니다."""
    H_path = os.path.join(output_folder, f'homography_{pair_name}.txt')
    if not os.path.exists(H_path):
        print(f"[ERROR] 호모그래피 파일을 찾을 수 없습니다: {H_path}")
        return None
    H = []
    with open(H_path, 'r') as f:
        for line in f:
            row = list(map(float, line.strip().split()))
            H.append(row)
    H = np.array(H)
    print(f"[DEBUG] Loaded homography for pair {pair_name}:")
    print(H)
    return H

def compute_panorama_size(images, homographies, central_idx):
    """
    모든 이미지의 호모그래피를 적용하여 파노라마 캔버스의 크기를 계산합니다.
    """
    print("[DEBUG] Computing panorama size.")
    corners = []
    for idx, (file_name, img) in enumerate(images):
        h, w = img.shape[:2]
        corner_points = np.array([
            [0, 0],
            [w, 0],
            [w, h],
            [0, h]
        ], dtype=np.float32).reshape(-1, 1, 2)
        
        # 누적 호모그래피 계산
        H_total = homographies.get(idx, np.eye(3))
        
        transformed_corners = cv2.perspectiveTransform(corner_points, H_total)
        corners.append(transformed_corners)
    
    all_corners = np.vstack(corners)
    min_x, min_y = np.min(all_corners, axis=0).flatten().astype(int)
    max_x, max_y = np.max(all_corners, axis=0).flatten().astype(int)
    
    print(f"[DEBUG] Panorama size: min_x={min_x}, min_y={min_y}, max_x={max_x}, max_y={max_y}")
    return min_x, min_y, max_x, max_y

def create_panorama(images, homographies, central_idx):
    """
    호모그래피 행렬을 사용하여 이미지를 파노라마로 합성합니다.
    """
    print("[DEBUG] Creating panorama.")
    # 파노라마 캔버스 크기 계산
    min_x, min_y, max_x, max_y = compute_panorama_size(images, homographies, central_idx)
    width = max_x - min_x
    height = max_y - min_y
    print(f"[DEBUG] Panorama dimensions: width={width}, height={height}")
    
    # 파노라마 캔버스 초기화
    panorama = np.zeros((height, width, 3), dtype=np.float32)
    count = np.zeros((height, width, 3), dtype=np.float32)
    
    for idx, (file_name, img) in enumerate(tqdm(images, desc="Warping Images")):
        H = homographies.get(idx, np.eye(3))
        # 변환된 이미지 위치 조정
        H_adjusted = H.copy()
        H_adjusted[0, 2] -= min_x
        H_adjusted[1, 2] -= min_y
        print(f"[DEBUG] Applying adjusted homography for image {file_name}:")
        print(H_adjusted)
        
        warped_img = cv2.warpPerspective(img, H_adjusted, (width, height))
        
        # 마스크 생성 (이미지가 있는 곳은 1, 없는 곳은 0)
        mask = (warped_img > 0).astype(np.float32)
        
        # 파노라마 이미지에 이미지 추가 및 카운트 증가
        panorama += warped_img * mask
        count += mask
    
    # 블렌딩: 평균을 내어 부드럽게 합성
    print("[DEBUG] Blending images to create panorama.")
    panorama /= np.maximum(count, 1)
    panorama = panorama.astype(np.uint8)
    
    return panorama

def save_panorama(panorama, output_path):
    """파노라마 이미지를 저장합니다."""
    cv2.imwrite(output_path, panorama)
    print(f"[INFO] 파노라마 이미지 저장 완료: {output_path}")

def extract_index(image_name):
    """
    이미지 이름에서 숫자 인덱스를 추출합니다.
    
    예: 'testimg01' -> 0
         'testimg02' -> 1
    """
    match = re.search(r'\d+', image_name)
    if match:
        return int(match.group()) - 1  # 0 기반 인덱스
    else:
        raise ValueError(f"이미지 이름에서 숫자를 추출할 수 없습니다: {image_name}")

def main_stitching():
    # "5. Homography" 폴더에서 호모그래피 불러오기
    homography_folder = '5. Homography'
    homography_files = os.listdir(homography_folder)
    
    # 호모그래피 파일 로드 및 매핑
    homographies = {}
    for file in homography_files:
        if file.startswith('homography_') and file.endswith('.txt'):
            pair = file[len('homography_'):-len('.txt')]
            try:
                img1_name, img2_name = pair.split('_to_')
            except ValueError:
                print(f"[ERROR] 호모그래피 파일 이름 형식이 올바르지 않습니다: {file}")
                continue
            try:
                img1_idx = extract_index(img1_name)
                img2_idx = extract_index(img2_name)
            except ValueError as e:
                print(f"[ERROR] {e}")
                continue
            H = load_homography(homography_folder, pair)
            if H is not None:
                homographies[img2_idx] = H  # img2_idx를 기준으로 호모그래피 저장
    
    # "2. Corner Detection" 폴더에서 이미지 불러오기
    corner_folder = '0. Image Input'
    file_names = os.listdir(corner_folder)
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    image_files = [f for f in file_names if os.path.splitext(f)[1].lower() in image_extensions]
    image_files.sort()  # 순차적으로 정렬 (testimg01, testimg02, ...)
    
    # 이미지 읽어서 리스트 구성
    corner_detected_images = []
    for f in image_files:
        path = os.path.join(corner_folder, f)
        img = cv2.imread(path)
        if img is not None:
            corner_detected_images.append((f, img))
        else:
            print(f"[WARNING] 이미지를 불러올 수 없습니다: {path}")
    
    print(f"[DEBUG] '2. Corner Detection' 폴더에서 {len(corner_detected_images)}장의 이미지를 불러왔습니다.")
    
    # 중앙 이미지 선택 (짝수 개일 경우 중앙에 가까운 이미지 선택)
    num_images = len(corner_detected_images)
    if num_images == 0:
        print("[ERROR] '2. Corner Detection' 폴더에 이미지가 없습니다.")
        return
    central_idx = num_images // 2
    print(f"[DEBUG] 중앙 이미지 인덱스: {central_idx} (파일명: {corner_detected_images[central_idx][0]})")
    
    # 호모그래피 누적 계산
    # 중앙 이미지를 기준으로 왼쪽과 오른쪽 이미지의 누적 호모그래피를 계산합니다.
    print("[DEBUG] Accumulating homographies relative to central image.")
    accumulated_homographies = {central_idx: np.eye(3)}
    
    # 왼쪽 이미지들에 대한 누적 호모그래피 계산
    for i in range(central_idx - 1, -1, -1):
        H = homographies.get(i + 1, None)  # homography from image i to image i+1
        if H is None:
            print(f"[WARNING] 호모그래피가 누락되었습니다: image{i+1} to image{i+2}")
            accumulated_homographies[i] = accumulated_homographies[i + 1]
        else:
            try:
                H_inv = np.linalg.inv(H)
                accumulated_homographies[i] = accumulated_homographies[i + 1] @ H_inv
                print(f"[DEBUG] Accumulated homography for image {i}:")
                print(accumulated_homographies[i])
            except np.linalg.LinAlgError:
                print(f"[ERROR] 호모그래피 행렬을 역행렬로 변환할 수 없습니다: image{i+1} to image{i+2}")
                accumulated_homographies[i] = accumulated_homographies[i + 1]
    
    # 오른쪽 이미지들에 대한 누적 호모그래피 계산
    for i in range(central_idx + 1, num_images):
        H = homographies.get(i, None)  # homography from image i-1 to image i
        if H is None:
            print(f"[WARNING] 호모그래피가 누락되었습니다: image{i} to image{i+1}")
            accumulated_homographies[i] = accumulated_homographies[i - 1]
        else:
            accumulated_homographies[i] = accumulated_homographies[i - 1] @ H
            print(f"[DEBUG] Accumulated homography for image {i}:")
            print(accumulated_homographies[i])
    
    # 파노라마 생성
    panorama = create_panorama(corner_detected_images, accumulated_homographies, central_idx)
    
    # 파노라마 저장
    panorama_output_folder = '6. Stitching'
    os.makedirs(panorama_output_folder, exist_ok=True)
    panorama_output_path = os.path.join(panorama_output_folder, 'result.jpg')
    save_panorama(panorama, panorama_output_path)
    
    print("[INFO] Stitching 작업이 완료되었습니다.")

if __name__ == "__main__":
    main_stitching()

[DEBUG] Loaded homography for pair testimg01.png_to_testimg02.png:
[[ 1.15162578e+00 -2.67242765e-02 -3.01262615e+02]
 [ 1.26472646e-01  1.08658805e+00 -4.45335266e+01]
 [ 4.03177516e-04 -1.85273776e-04  1.00000000e+00]]
[DEBUG] Loaded homography for pair testimg02.png_to_testimg03.png:
[[ 8.70928280e-01  5.00735176e-03 -2.23194570e+02]
 [-1.10589227e-01  1.01144635e+00  1.49760572e+01]
 [-4.46749591e-04  3.36650486e-04  1.00000000e+00]]
[DEBUG] Loaded homography for pair testimg03.png_to_testimg04.png:
[[ 1.57608099e+00  1.37933193e-01 -4.48845600e+02]
 [ 1.78278416e-01  1.86438202e+00 -1.57555557e+02]
 [ 6.02291946e-04  1.13938828e-03  1.00000000e+00]]
[DEBUG] Loaded homography for pair testimg04.png_to_testimg05.png:
[[ 1.23156106e+00 -2.15314762e-02 -3.20903875e+02]
 [ 1.40864596e-01  1.19975728e+00 -5.43948776e+01]
 [ 4.42235278e-04  9.36298656e-05  1.00000000e+00]]
[DEBUG] Loaded homography for pair testimg05.png_to_testimg06.png:
[[ 1.19168893e+00 -5.13424581e-02 -3.07628137e+02

Warping Images:   0%|                                                                                                       | 0/10 [00:00<?, ?it/s]

[DEBUG] Applying adjusted homography for image testimg01.png:
[[ 1.82244417e-01 -2.84646068e-01  1.10002519e+04]
 [-1.37683806e-01  3.86850710e-01  4.05057548e+03]
 [-4.16230308e-04 -2.93470870e-04  2.46017428e-01]]


Warping Images:  10%|█████████▌                                                                                     | 1/10 [00:03<00:34,  3.83s/it]

[DEBUG] Applying adjusted homography for image testimg02.png:
[[ 3.98145683e-01 -4.17222256e-01  1.09580248e+04]
 [-1.36011881e-01  4.36148308e-01  4.07482664e+03]
 [-4.17268893e-04 -3.53339066e-04  3.84481352e-01]]


...

[DEBUG] Applying adjusted homography for image testimg09.png:
[[-1.63681684e+00 -4.92874631e+01  1.78910350e+04]
 [-8.03858938e-01 -2.42056208e+01  7.77332164e+03]
 [-3.63737076e-03 -1.09527696e-01  1.65489667e+01]]


Warping Images:  90%|█████████████████████████████████████████████████████████████████████████████████████▌         | 9/10 [00:47<00:04,  4.93s/it]

[DEBUG] Applying adjusted homography for image testimg10.png:
[[-5.94967334e+01  9.19277171e+01  1.69815039e+04]
 [-2.92195069e+01  4.51467233e+01  7.32664081e+03]
 [-1.32214963e-01  2.04283816e-01  1.45277865e+01]]



arping Images: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:51<00:00,  5.10s/it]

[DEBUG] Blending images to create panorama.
[INFO] 파노라마 이미지 저장 완료: 6. Stitching\result.jpg
[INFO] Stitching 작업이 완료되었습니다.


In [39]:
def group_adjustment():
    """
    파노라마 이미지의 색상 균일화 및 노출 보정을 수행하고 결과를 저장
    """
    print("\n[INFO] #7. Group Adjustment 단계 시작")
    
    # 파노라마 이미지 로드
    panorama_path = os.path.join('6. Stitching', 'result.jpg')
    if not os.path.exists(panorama_path):
        print(f"[ERROR] 파노라마 이미지 파일을 찾을 수 없습니다: {panorama_path}")
        return
    
    panorama = cv2.imread(panorama_path)
    if panorama is None:
        print(f"[ERROR] 파노라마 이미지를 불러올 수 없습니다: {panorama_path}")
        return
    print(f"[DEBUG] 파노라마 이미지 로드 완료: {panorama_path}, 크기: {panorama.shape}")
    
    # 색상 균일화 및 노출 보정
    print("[DEBUG] 색상 균일화 및 노출 보정 시작")
    # 각 채널별로 평균과 표준 편차 계산
    channels = cv2.split(panorama)
    adjusted_channels = []
    for idx, channel in enumerate(channels):
        mean = np.mean(channel)
        std = np.std(channel)
        print(f"[DEBUG] 채널 {idx} 평균: {mean:.2f}, 표준 편차: {std:.2f}")
        # Z-score 정규화
        normalized = (channel - mean) / (std + 1e-8)
        # 다시 평균과 표준 편차를 맞추어 재조정
        target_mean = np.mean(channel)
        target_std = np.std(channel)
        adjusted = normalized * target_std + target_mean
        # 픽셀 값 클리핑
        adjusted = np.clip(adjusted, 0, 255).astype(np.uint8)
        adjusted_channels.append(adjusted)
        print(f"[DEBUG] 채널 {idx} 정규화 및 재조정 완료")
    
    # 채널 병합
    adjusted_panorama = cv2.merge(adjusted_channels)
    print("[DEBUG] 색상 균일화 및 노출 보정 완료")
    
    # 결과 저장
    output_folder = '7. Group Adjustment'
    os.makedirs(output_folder, exist_ok=True)
    result_path = os.path.join(output_folder, 'result.jpg')
    cv2.imwrite(result_path, adjusted_panorama)
    print(f"[INFO] 색상 균일화 및 노출 보정된 파노라마 이미지 저장 완료: {result_path}")

if __name__ == "__main__":
    # 이전 단계 코드는 이미 실행되었다고 가정
    # 여기서 Group Adjustment을 수행합니다.
    group_adjustment()
    print("[INFO] #7. Group Adjustment 작업이 완료되었습니다.")


[INFO] #7. Group Adjustment 단계 시작
[DEBUG] 파노라마 이미지 로드 완료: 6. Stitching\result.jpg, 크기: (7246, 24369, 3)
[DEBUG] 색상 균일화 및 노출 보정 시작
[DEBUG] 채널 0 평균: 3.57, 표준 편차: 26.34
[DEBUG] 채널 0 정규화 및 재조정 완료
[DEBUG] 채널 1 평균: 3.85, 표준 편차: 26.98
[DEBUG] 채널 1 정규화 및 재조정 완료
[DEBUG] 채널 2 평균: 3.65, 표준 편차: 26.35
[DEBUG] 채널 2 정규화 및 재조정 완료
[DEBUG] 색상 균일화 및 노출 보정 완료
[INFO] 색상 균일화 및 노출 보정된 파노라마 이미지 저장 완료: 7. Group Adjustment\result.jpg
[INFO] #7. Group Adjustment 작업이 완료되었습니다.


In [41]:
def load_image(image_path):
    """이미지를 불러옵니다."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
    return img

def compute_luminance(image):
    """이미지의 각 픽셀에 대한 luminance를 계산합니다."""
    # OpenCV는 BGR 형식으로 이미지를 로드하므로, RGB로 변환
    B = image[:, :, 0].astype(np.float32)
    G = image[:, :, 1].astype(np.float32)
    R = image[:, :, 2].astype(np.float32)
    # ITU-R BT.709 표준에 따른 luminance 계산
    luminance = 0.2126 * R + 0.7152 * G + 0.0722 * B
    return luminance

def compute_average_log_luminance(luminance, epsilon=1e-4):
    """전체 이미지의 평균 로그 luminance를 계산합니다."""
    log_luminance = np.log(luminance + epsilon)
    average_log_luminance = np.exp(np.mean(log_luminance))
    return average_log_luminance

def tone_map_image(image, a=0.18):
    """
    Reinhard의 글로벌 Tone Mapping 연산자를 적용하여 이미지를 Tone Mapping합니다.
    
    Parameters:
        image: 입력 이미지 (BGR 형식, uint8)
        a: key 값 (일반적으로 0.18 사용)
    
    Returns:
        tone_mapped_image: Tone Mapping이 적용된 이미지 (BGR 형식, uint8)
    """
    # Step 1: Luminance 계산
    luminance = compute_luminance(image)
    
    # Step 2: 평균 로그 luminance 계산
    L_avg = compute_average_log_luminance(luminance)
    print(f"[DEBUG] 평균 로그 luminance: {L_avg:.4f}")
    
    # Step 3: Tone Mapping 적용
    L_mapped = (a / L_avg) * luminance / (1 + (a / L_avg) * luminance)
    
    # Step 4: 색상 정보 보존을 위해 RGB 채널 조정
    # 방정식: C_mapped = C * (L_mapped / L)
    # 단, L이 0인 경우를 방지하기 위해 epsilon 추가
    epsilon = 1e-4
    L = luminance + epsilon
    B = image[:, :, 0].astype(np.float32)
    G = image[:, :, 1].astype(np.float32)
    R = image[:, :, 2].astype(np.float32)
    
    B_mapped = B * (L_mapped / L)
    G_mapped = G * (L_mapped / L)
    R_mapped = R * (L_mapped / L)
    
    # Step 5: 합성된 채널을 uint8로 변환
    tone_mapped_image = np.stack([B_mapped, G_mapped, R_mapped], axis=2)
    tone_mapped_image = np.clip(tone_mapped_image * 255.0, 0, 255).astype(np.uint8)
    
    return tone_mapped_image

def save_image(image, output_path):
    """이미지를 저장합니다."""
    cv2.imwrite(output_path, image)
    print(f"Tone Mapped 이미지 저장 완료: {output_path}")

def main_tone_mapping():
    """
    Tone Mapping 단계를 수행합니다.
    
    Steps:
        1. '7. Group Adjustment' 폴더에서 그룹 조정된 파노라마 이미지 불러오기
        2. Tone Mapping 적용
        3. '8. Tone Mapping' 폴더에 저장
    """
    # 그룹 조정된 파노라마 이미지 경로
    group_adjusted_panorama_path = os.path.join('7. Group Adjustment', 'result.jpg')
    
    # Tone Mapping 결과를 저장할 폴더
    tone_mapping_output_folder = '8. Tone Mapping'
    os.makedirs(tone_mapping_output_folder, exist_ok=True)
    
    # 1. 파노라마 이미지 불러오기
    panorama_img = load_image(group_adjusted_panorama_path)
    if panorama_img is None:
        print("Tone Mapping을 수행할 파노라마 이미지를 불러올 수 없습니다.")
        return
    
    print(f"[DEBUG] 파노라마 이미지 로드 완료: {group_adjusted_panorama_path}, 크기: {panorama_img.shape}")
    
    # 2. Tone Mapping 적용
    tone_mapped_img = tone_map_image(panorama_img, a=0.18)
    print("[DEBUG] Tone Mapping 적용 완료.")
    
    # 3. Tone Mapping 결과 저장
    tone_mapped_output_path = os.path.join(tone_mapping_output_folder, 'result.jpg')
    save_image(tone_mapped_img, tone_mapped_output_path)
    
    print("Tone Mapping 작업이 완료되었습니다.")

if __name__ == "__main__":
    # 기존 단계 실행
    # ...
    # Point Matching 작업이 모두 완료되었습니다.
    
    # Homography 계산 단계는 이미 실행되었다고 가정
    
    # Group Adjustment 단계 실행
    # main_group_adjustment()  # 이미 실행되었다고 가정
    
    # Tone Mapping 단계 실행
    main_tone_mapping()

[DEBUG] 파노라마 이미지 로드 완료: 7. Group Adjustment\result.jpg, 크기: (7246, 24369, 3)
[DEBUG] 평균 로그 luminance: 0.0001
[DEBUG] Tone Mapping 적용 완료.
Tone Mapped 이미지 저장 완료: 8. Tone Mapping\result.jpg
Tone Mapping 작업이 완료되었습니다.
